In [ ]:
# [실험 2] Dropout 모델 정의
# 과적합을 방지하기 위해 nn.Dropout 레이어를 추가합니다.
class MLP_Dropout(nn.Module):
    def __init__(self, input_size=784, hidden_size=100, num_classes=10, dropout_prob=0.5):
        super(MLP_Dropout, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_prob),  # Dropout 레이어 추가
            nn.Linear(hidden_size, num_classes)
        )
    
    def forward(self, x):
        return self.layers(x)

# ====================================================================
# [실험 2] 훈련 루프 실행 (Dropout 모델, LR=1e-3)
# ====================================================================

# 하이퍼파라미터 재설정 (기본 LR인 1e-3으로 재설정)
learning_rate = 1e-3
nb_epochs = 3

# Dropout 모델로 초기화
model_exp2 = MLP_Dropout().to(device)
criterion_exp2 = nn.CrossEntropyLoss()
optimizer_exp2 = optim.Adam(model_exp2.parameters(), lr=learning_rate)

# 훈련 과정 추적을 위한 리스트 초기화
train_losses_exp2 = []
train_accuracies_exp2 = []
test_accuracies_exp2 = []

print(f"\n=== [실험 2] Dropout 모델 (LR={learning_rate}) 재훈련 시작 ===")
print(f"새 모델: MLP_Dropout, 에포크 수: {nb_epochs}\n")

for epoch in range(nb_epochs):
    model_exp2.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for batch_idx, batch in enumerate(train_loader):
        imgs = batch["image"].to(device)
        labels = batch["label"].to(device)

        optimizer_exp2.zero_grad()
        outputs = model_exp2(imgs)
        loss = criterion_exp2(outputs, labels)
        loss.backward()
        optimizer_exp2.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        if (batch_idx + 1) % 100 == 0:
            current_loss = running_loss / (batch_idx + 1)
            current_acc = 100 * correct_train / total_train
            print(f"Epoch [{epoch+1}/{nb_epochs}], Batch [{batch_idx+1}/{len(train_loader)}]")
            print(f"  Loss: {current_loss:.4f}, Train Acc: {current_acc:.2f}%")

    epoch_loss = running_loss / len(train_loader)
    epoch_train_acc = 100 * correct_train / total_train
    train_losses_exp2.append(epoch_loss)
    train_accuracies_exp2.append(epoch_train_acc)

    print(f"\nEpoch [{epoch+1}/{nb_epochs}] 훈련 완료:")
    print(f"  평균 Loss: {epoch_loss:.4f}")
    print(f"  훈련 정확도: {epoch_train_acc:.2f}%")

    model_exp2.eval()
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for batch in test_loader:
            imgs = batch["image"].to(device)
            labels = batch["label"].to(device)
            outputs = model_exp2(imgs)
            _, predicted = torch.max(outputs, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    test_acc = 100 * correct_test / total_test
    test_accuracies_exp2.append(test_acc)
    print(f"  테스트 정확도: {test_acc:.2f}%")
    print("-" * 60)

print(f"\n=== 실험 2 훈련 완료 (Dropout) ===")
print(f"최종 테스트 정확도: {test_accuracies_exp2[-1]:.2f}%")